In [2]:
import pandas as pd
import numpy as np
import matplotlib

In [3]:
#conda install pycountry

In [4]:
import pycountry

In [5]:
productivity = '~/uniwork/ads/ADS-European-Productivity/Datasets/cleaned_Productivity.csv'
employment = '~/uniwork/ads/ADS-European-Productivity/Datasets/employment_growth.csv'

In [6]:
prod_pd = pd.read_csv(productivity)
emp_pd = pd.read_csv(employment, header=0, nrows=None)


In [7]:
#making data look nice so I can make the 2 datasets work with each other
reformattedData = pd.pivot_table(emp_pd, index='geo', columns='TIME_PERIOD', values='OBS_VALUE')
emp_pd_reformatted = reformattedData.reset_index().rename_axis(None, axis=1)

In [8]:
# drops aggregated values 

indicesToDrop = emp_pd_reformatted[((emp_pd_reformatted.geo == 'EA19') | 
                                 (emp_pd_reformatted.geo == 'EU27_2020') | 
                                 (emp_pd_reformatted.geo == 'EA20') |
                                 (emp_pd_reformatted.geo == 'EU28') |
                                   (emp_pd_reformatted.geo == 'EA'))].index
emp_pd_reformatted.drop(indicesToDrop, inplace=True)

In [9]:
#replace NaN value with column mean
emp_pd_no_nan = emp_pd_reformatted.fillna(emp_pd_reformatted.mean())

/tmp/ipykernel_82382/2953700504.py:2: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  emp_pd_no_nan = emp_pd_reformatted.fillna(emp_pd_reformatted.mean())


In [10]:
#change country code from 2 letter to 3 letter
emp_pd_3letter = emp_pd_no_nan
for index, row in emp_pd_3letter.iterrows():
    if row.geo == 'EL':
        row.geo = 'GR'
    if row.geo == 'UK':
        row.geo = 'GB'
#     print(index)
    emp_pd_3letter.at[index, 'geo'] = pycountry.countries.get(alpha_2 = row.geo).alpha_3
    
    

In [11]:
#list of countries in employment dataset
countries = emp_pd_3letter.geo
countries = np.array(countries)

In [12]:
#get rid of countries in productivity dataset that are not in employment growth
for index, row in prod_pd.iterrows():
    if row.LOCATION not in countries:
        prod_pd.drop(index, inplace=True)

In [13]:
#list of countries in productivity dataset
prod_countries = np.array(prod_pd.LOCATION)

In [14]:
#get rid of countries in employment dataset that are not in productivity
for index, row in emp_pd_3letter.iterrows():
    if row.geo not in prod_countries:
        emp_pd_3letter.drop(index, inplace=True)

In [15]:
#drop 2000 to 2004 in productivity dataset
#prod_pd.drop('2000', axis = 1, inplace = True)
prod_pd.drop('2001', axis = 1, inplace = True)
prod_pd.drop('2002', axis = 1, inplace = True)
prod_pd.drop('2003', axis = 1, inplace = True)
prod_pd.drop('2004', axis = 1, inplace = True)

In [16]:
prod_pd.drop('mean', axis = 1, inplace = True)


In [17]:
prod_pd.head()

,LOCATION,2000,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
1,AUT,55.825179,60.632340,62.218785,63.814049,63.814456,63.363388,64.291488,64.947960,65.645538,66.080679,66.327394,67.267651,67.184974,67.999464,68.309455,68.309868,69.970148,69.634603
2,BEL,63.089170,67.287921,67.637137,68.887444,68.464336,68.020751,69.053627,68.581978,68.923207,69.517696,70.513657,71.604653,71.674769,71.534022,71.681631,72.259271,74.541737,73.607184
3,BGR,14.861829,17.965425,18.618248,19.229785,19.475667,19.717737,20.847786,21.781488,22.497211,22.460713,22.607390,23.296069,23.897609,24.130056,24.784139,25.687153,25.775778,26.559335
5,CHE,61.437020,64.693828,66.315161,67.595243,68.253612,66.723475,68.620315,68.227615,68.494780,69.759903,70.447419,69.982344,70.431631,71.561236,73.212853,73.619376,74.655861,75.909195
9,CZE,25.998047,32.765277,34.781571,36.256014,36.139167,35.303035,36.128328,36.734795,36.886973,37.004724,37.432375,39.390083,39.251558,40.595969,41.161265,42.272668,43.289836,43.487932


In [18]:
emp_pd_3letter.drop(2022,axis = 1, inplace = True)

In [19]:
emp_pd_3letter.head()

,geo,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AUT,1.2,1.8,1.8,1.9,-0.5,0.8,1.6,1.0,0.3,1.0,0.6,1.3,1.6,1.7,1.1,-1.6,2.0
1,BEL,1.4,1.1,1.7,1.8,-0.2,0.6,1.3,0.4,-0.3,0.4,0.9,1.3,1.6,1.5,1.6,0.1,1.9
2,BGR,2.7,3.3,3.2,2.4,-1.7,-3.9,-2.2,-2.5,-0.4,0.4,0.4,0.5,1.8,-0.1,0.3,-2.3,0.2
3,CHE,0.7,2.2,2.6,2.4,0.5,0.3,2.6,1.8,1.2,1.9,1.5,1.4,0.9,1.0,0.7,-0.5,0.6
5,CZE,1.9,1.3,2.1,2.2,-1.8,-1.0,-0.3,0.4,0.3,0.6,1.4,1.6,1.5,1.3,0.2,-1.7,0.4


In [20]:
prod_timeseries = pd.melt(prod_pd, id_vars = ['LOCATION'], var_name = 'YEAR', value_name= 'PRODUCTIVITY')

In [21]:
emp_timeseries = pd.melt(emp_pd_3letter, id_vars = ['geo'], var_name = 'YEAR', value_name= 'EMPLOYMENT')

In [22]:
data_pd = prod_timeseries
data_pd['EMPLOYMENT'] = emp_timeseries['EMPLOYMENT']

In [23]:
data_pd

,LOCATION,YEAR,PRODUCTIVITY,EMPLOYMENT
0,AUT,2000,55.825179,1.2
1,BEL,2000,63.089170,1.4
2,BGR,2000,14.861829,2.7
3,CHE,2000,61.437020,0.7
4,CZE,2000,25.998047,1.9
...,...,...,...,...
517,PRT,2021,40.508787,NaN
518,ROU,2021,37.514761,NaN
519,SVK,2021,48.091287,NaN
520,SVN,2021,47.075129,NaN


In [24]:
data_pd['PRODUCTIVITY'].corr(data_pd['EMPLOYMENT'])

-0.005835359179973838

In [25]:
data_pd.groupby('LOCATION')[['PRODUCTIVITY', 'EMPLOYMENT']].corr()

PRODUCTIVITY  EMPLOYMENT
LOCATION                                       
AUT      PRODUCTIVITY      1.000000   -0.140046
         EMPLOYMENT       -0.140046    1.000000
BEL      PRODUCTIVITY      1.000000    0.157325
         EMPLOYMENT        0.157325    1.000000
BGR      PRODUCTIVITY      1.000000   -0.351455
         EMPLOYMENT       -0.351455    1.000000
CHE      PRODUCTIVITY      1.000000   -0.378044
         EMPLOYMENT       -0.378044    1.000000
CZE      PRODUCTIVITY      1.000000   -0.300400
         EMPLOYMENT       -0.300400    1.000000
DEU      PRODUCTIVITY      1.000000   -0.007790
         EMPLOYMENT       -0.007790    1.000000
DNK      PRODUCTIVITY      1.000000    0.248435
         EMPLOYMENT        0.248435    1.000000
ESP      PRODUCTIVITY      1.000000    0.113942
         EMPLOYMENT        0.113942    1.000000
EST      PRODUCTIVITY      1.000000    0.115207
         EMPLOYMENT        0.115207    1.000000
FIN      PRODUCTIVITY      1.000000   -0.208065
         EMPLOYMENT       -0.208065    1.000000
FRA      PRODUCTIVITY      1.000000    0.040298
         EMPLOYMENT        0.040298    1.000000
GBR      PRODUCTIVITY      1.000000    0.149823
         EMPLOYMENT        0.149823    1.000000
GRC      PRODUCTIVITY      1.000000   -0.229277
         EMPLOYMENT       -0.229277    1.000000
HRV      PRODUCTIVITY      1.000000    0.423361
         EMPLOYMENT        0.423361    1.000000
HUN      PRODUCTIVITY      1.000000   -0.187772
         EMPLOYMENT       -0.187772    1.000000
IRL      PRODUCTIVITY      1.000000   -0.129099
         EMPLOYMENT       -0.129099    1.000000
ISL      PRODUCTIVITY      1.000000   -0.206007
         EMPLOYMENT       -0.206007    1.000000
ITA      PRODUCTIVITY      1.000000    0.448419
         EMPLOYMENT        0.448419    1.000000
LTU      PRODUCTIVITY      1.000000   -0.114206
         EMPLOYMENT       -0.114206    1.000000
LUX      PRODUCTIVITY      1.000000    0.100454
         EMPLOYMENT        0.100454    1.000000
LVA      PRODUCTIVITY      1.000000    0.121624
         EMPLOYMENT        0.121624    1.000000
NLD      PRODUCTIVITY      1.000000   -0.226659
         EMPLOYMENT       -0.226659    1.000000
NOR      PRODUCTIVITY      1.000000    0.221237
         EMPLOYMENT        0.221237    1.000000
POL      PRODUCTIVITY      1.000000    0.291495
         EMPLOYMENT        0.291495    1.000000
PRT      PRODUCTIVITY      1.000000    0.091263
         EMPLOYMENT        0.091263    1.000000
ROU      PRODUCTIVITY      1.000000   -0.028827
         EMPLOYMENT       -0.028827    1.000000
SVK      PRODUCTIVITY      1.000000    0.214578
         EMPLOYMENT        0.214578    1.000000
SVN      PRODUCTIVITY      1.000000   -0.245874
         EMPLOYMENT       -0.245874    1.000000
SWE      PRODUCTIVITY      1.000000    0.011060
         EMPLOYMENT        0.011060    1.000000

In [176]:
esi = "~/uniwork/ads/ADS-European-Productivity/Datasets/ESI_CLEAN.csv"
esi_df = pd.read_csv(esi)
esi_df.columns=esi_df.iloc[0]
esi_df.drop(0, inplace=True) 
geo = esi_df['Month-Year']


# esi_df.set_index('geo', drop=True, append=False, inplace=True, verify_integrity=False)

In [177]:
esi_df = esi_df.drop('Month-Year', axis = 1)

In [178]:
esi_df = esi_df.astype(np.float64)

In [179]:

esi_df['geo'] = geo
esi_df['2005'] = esi_df.filter(regex='-05').mean(axis=1)
esi_df['2006'] = esi_df.filter(regex='-06').mean(axis=1)
esi_df['2007'] = esi_df.filter(regex='-07').mean(axis=1)
esi_df['2008'] = esi_df.filter(regex='-08').mean(axis=1)
esi_df['2009'] = esi_df.filter(regex='-09').mean(axis=1)
esi_df['2010'] = esi_df.filter(regex='-10').mean(axis=1)
esi_df['2011'] = esi_df.filter(regex='-11').mean(axis=1)
esi_df['2012'] = esi_df.filter(regex='-12').mean(axis=1)
esi_df['2013'] = esi_df.filter(regex='-13').mean(axis=1)
esi_df['2014'] = esi_df.filter(regex='-14').mean(axis=1)
esi_df['2015'] = esi_df.filter(regex='-15').mean(axis=1)
esi_df['2016'] = esi_df.filter(regex='-16').mean(axis=1)
esi_df['2017'] = esi_df.filter(regex='-17').mean(axis=1)
esi_df['2018'] = esi_df.filter(regex='-18').mean(axis=1)
esi_df['2019'] = esi_df.filter(regex='-19').mean(axis=1)
esi_df['2020'] = esi_df.filter(regex='-20').mean(axis=1)
esi_df['2021'] = esi_df.filter(regex='-21').mean(axis=1)

esi_df = esi_df[['geo','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019','2020','2021']]

In [180]:
esi_df.drop([1, 2],axis = 0, inplace=True)

In [181]:
esi_df

,geo,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
3,BE.ESI,100.966667,111.616667,112.150000,95.008333,80.416667,102.525000,101.850000,91.033333,94.850000,100.600000,100.675000,102.900000,105.233333,105.533333,97.800000,86.033333,111.150000
4,BG.ESI,107.725000,111.033333,117.575000,114.716667,89.433333,89.883333,92.066667,91.883333,92.166667,95.741667,100.316667,102.741667,106.591667,106.958333,102.550000,82.658333,96.183333
5,CZ.ESI,108.166667,113.358333,114.400000,102.775000,80.383333,95.625000,94.516667,87.833333,89.633333,98.725000,103.016667,105.358333,106.741667,107.808333,103.783333,84.958333,94.008333
6,DK.ESI,103.783333,113.016667,105.891667,88.466667,83.983333,108.341667,101.016667,92.725000,97.091667,100.566667,101.708333,101.908333,108.950000,109.366667,102.616667,86.883333,109.908333
7,DE.ESI,93.783333,103.975000,105.808333,92.833333,82.025000,104.333333,107.500000,99.500000,101.625000,103.866667,105.191667,105.533333,110.900000,111.558333,102.475000,91.208333,110.016667
8,EE.ESI,111.216667,116.666667,108.391667,87.825000,75.525000,98.083333,102.341667,99.041667,99.425000,98.433333,95.333333,99.375000,105.033333,103.741667,97.966667,86.975000,105.058333
9,EL.ESI,102.241667,108.100000,110.291667,101.166667,87.241667,85.383333,83.458333,83.016667,92.641667,101.016667,91.141667,94.700000,98.150000,102.866667,105.633333,95.475000,106.116667
10,ES.ESI,104.841667,104.125000,103.350000,83.741667,79.866667,89.575000,89.391667,85.858333,90.650000,99.858333,107.750000,106.033333,109.216667,108.025000,104.675000,89.841667,105.116667
11,FR.ESI,101.966667,107.641667,110.800000,93.575000,81.416667,99.016667,100.066667,89.025000,88.133333,91.216667,96.566667,99.616667,107.025000,105.483333,102.491667,89.333333,107.725000
12,CY.ESI,101.708333,103.475000,109.350000,106.533333,91.458333,94.983333,87.766667,81.533333,82.208333,93.525000,99.375000,106.508333,110.841667,111.241667,109.858333,88.925000,99.166667


In [182]:
countries = np.asarray(set(data_pd['LOCATION']))
countries

array({'DEU', 'BGR', 'HUN', 'LTU', 'POL', 'ESP', 'CZE', 'IRL', 'NLD', 'SWE', 'PRT', 'SVK', 'FIN', 'AUT', 'NOR', 'LVA', 'DNK', 'FRA', 'GRC', 'ISL', 'CHE', 'SVN', 'HRV', 'EST', 'ITA', 'GBR', 'LUX', 'BEL', 'ROU'},
      dtype=object)

In [183]:

for index, row in esi_df.iterrows():
    if row.geo == 'EL.ESI':
        row.geo = 'GR.ESI'
    if row.geo == 'UK.ESI':
        row.geo = 'GB.ESI'
    
    esi_df.at[index, 'geo'] = pycountry.countries.get(alpha_2 = row.geo[:-4]).alpha_3
    
    

In [184]:
esi_df.set_index('geo', drop=True, append=False, inplace=True, verify_integrity=False)

In [186]:
esi_df = esi_df.sort_index()

In [188]:
esi_df

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
geo,,,,,,,,,,,,,,,,,
AUT,97.791667,110.241667,110.925000,92.183333,79.966667,100.575000,99.650000,94.650000,96.058333,97.025000,97.641667,101.350000,112.033333,113.000000,103.416667,87.316667,111.916667
BEL,100.966667,111.616667,112.150000,95.008333,80.416667,102.525000,101.850000,91.033333,94.850000,100.600000,100.675000,102.900000,105.233333,105.533333,97.800000,86.033333,111.150000
BGR,107.725000,111.033333,117.575000,114.716667,89.433333,89.883333,92.066667,91.883333,92.166667,95.741667,100.316667,102.741667,106.591667,106.958333,102.550000,82.658333,96.183333
CYP,101.708333,103.475000,109.350000,106.533333,91.458333,94.983333,87.766667,81.533333,82.208333,93.525000,99.375000,106.508333,110.841667,111.241667,109.858333,88.925000,99.166667
CZE,108.166667,113.358333,114.400000,102.775000,80.383333,95.625000,94.516667,87.833333,89.633333,98.725000,103.016667,105.358333,106.741667,107.808333,103.783333,84.958333,94.008333
DEU,93.783333,103.975000,105.808333,92.833333,82.025000,104.333333,107.500000,99.500000,101.625000,103.866667,105.191667,105.533333,110.900000,111.558333,102.475000,91.208333,110.016667
DNK,103.783333,113.016667,105.891667,88.466667,83.983333,108.341667,101.016667,92.725000,97.091667,100.566667,101.708333,101.908333,108.950000,109.366667,102.616667,86.883333,109.908333
ESP,104.841667,104.125000,103.350000,83.741667,79.866667,89.575000,89.391667,85.858333,90.650000,99.858333,107.750000,106.033333,109.216667,108.025000,104.675000,89.841667,105.116667
EST,111.216667,116.666667,108.391667,87.825000,75.525000,98.083333,102.341667,99.041667,99.425000,98.433333,95.333333,99.375000,105.033333,103.741667,97.966667,86.975000,105.058333


In [192]:
esi_df.drop(['CYP', 'MLT'], inplace=True)

In [172]:
emp_pd_3letter.set_index('geo', drop=True, append=False, inplace=True, verify_integrity=False)

In [193]:
esi_df

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
geo,,,,,,,,,,,,,,,,,
AUT,97.791667,110.241667,110.925000,92.183333,79.966667,100.575000,99.650000,94.650000,96.058333,97.025000,97.641667,101.350000,112.033333,113.000000,103.416667,87.316667,111.916667
BEL,100.966667,111.616667,112.150000,95.008333,80.416667,102.525000,101.850000,91.033333,94.850000,100.600000,100.675000,102.900000,105.233333,105.533333,97.800000,86.033333,111.150000
BGR,107.725000,111.033333,117.575000,114.716667,89.433333,89.883333,92.066667,91.883333,92.166667,95.741667,100.316667,102.741667,106.591667,106.958333,102.550000,82.658333,96.183333
CZE,108.166667,113.358333,114.400000,102.775000,80.383333,95.625000,94.516667,87.833333,89.633333,98.725000,103.016667,105.358333,106.741667,107.808333,103.783333,84.958333,94.008333
DEU,93.783333,103.975000,105.808333,92.833333,82.025000,104.333333,107.500000,99.500000,101.625000,103.866667,105.191667,105.533333,110.900000,111.558333,102.475000,91.208333,110.016667
DNK,103.783333,113.016667,105.891667,88.466667,83.983333,108.341667,101.016667,92.725000,97.091667,100.566667,101.708333,101.908333,108.950000,109.366667,102.616667,86.883333,109.908333
ESP,104.841667,104.125000,103.350000,83.741667,79.866667,89.575000,89.391667,85.858333,90.650000,99.858333,107.750000,106.033333,109.216667,108.025000,104.675000,89.841667,105.116667
EST,111.216667,116.666667,108.391667,87.825000,75.525000,98.083333,102.341667,99.041667,99.425000,98.433333,95.333333,99.375000,105.033333,103.741667,97.966667,86.975000,105.058333
FIN,107.150000,111.800000,112.375000,94.541667,83.341667,105.691667,102.075000,93.483333,91.508333,89.808333,91.716667,97.508333,108.541667,108.841667,99.391667,85.591667,109.041667


In [194]:
emp_pd_3letter = emp_pd_3letter.sort_index()
emp_pd_3letter.drop(['CHE', 'GBR', 'HRV', 'IRL', 'ISL','ITA', 'NOR'],inplace=True)
emp_pd_3letter

,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
geo,,,,,,,,,,,,,,,,,
AUT,1.2,1.8,1.8,1.9,-0.5,0.8,1.6,1.0,0.3,1.0,0.6,1.3,1.6,1.7,1.1,-1.6,2.0
BEL,1.4,1.1,1.7,1.8,-0.2,0.6,1.3,0.4,-0.3,0.4,0.9,1.3,1.6,1.5,1.6,0.1,1.9
BGR,2.7,3.3,3.2,2.4,-1.7,-3.9,-2.2,-2.5,-0.4,0.4,0.4,0.5,1.8,-0.1,0.3,-2.3,0.2
CZE,1.9,1.3,2.1,2.2,-1.8,-1.0,-0.3,0.4,0.3,0.6,1.4,1.6,1.5,1.3,0.2,-1.7,0.4
DEU,-0.1,0.7,1.7,1.4,0.2,0.4,1.2,1.1,0.8,0.9,0.9,1.2,1.4,1.4,0.9,-0.8,0.1
DNK,1.4,2.3,2.3,1.2,-3.1,-2.3,0.0,-0.7,0.0,0.9,1.4,1.7,1.5,1.5,1.4,-1.1,2.4
ESP,4.2,4.1,3.2,0.1,-6.3,-1.7,-2.5,-4.0,-2.4,1.0,2.8,2.1,2.6,2.2,2.6,-4.2,2.5
EST,2.3,4.9,0.2,-0.2,-10.2,-4.9,6.5,1.6,1.2,0.8,2.9,0.3,2.7,0.9,1.3,-2.7,0.1
FIN,1.5,1.9,2.2,2.2,-2.5,-0.6,1.6,0.9,-0.8,-0.4,-0.1,0.5,1.0,2.5,1.8,-1.9,2.7


In [204]:
esi_df = pd.melt(esi_df, ignore_index=False, var_name = 'YEAR', value_name= 'ESI')
emp_pd_3letter = pd.melt(emp_pd_3letter, ignore_index=False, var_name = 'YEAR', value_name= 'EMPLOYMENT')

In [207]:
data_pd = esi_df
data_pd['EMPLOYMENT'] = emp_pd_3letter['EMPLOYMENT']

In [209]:
data_pd.groupby('geo')[['ESI', 'EMPLOYMENT']].corr()

ESI  EMPLOYMENT
geo                                 
AUT ESI         1.000000    0.749984
    EMPLOYMENT  0.749984    1.000000
BEL ESI         1.000000    0.698389
    EMPLOYMENT  0.698389    1.000000
BGR ESI         1.000000    0.888601
    EMPLOYMENT  0.888601    1.000000
CZE ESI         1.000000    0.835189
    EMPLOYMENT  0.835189    1.000000
DEU ESI         1.000000    0.519106
    EMPLOYMENT  0.519106    1.000000
DNK ESI         1.000000    0.613398
    EMPLOYMENT  0.613398    1.000000
ESP ESI         1.000000    0.890864
    EMPLOYMENT  0.890864    1.000000
EST ESI         1.000000    0.728284
    EMPLOYMENT  0.728284    1.000000
FIN ESI         1.000000    0.773602
    EMPLOYMENT  0.773602    1.000000
FRA ESI         1.000000    0.840612
    EMPLOYMENT  0.840612    1.000000
GRC ESI         1.000000    0.779756
    EMPLOYMENT  0.779756    1.000000
HUN ESI         1.000000    0.788095
    EMPLOYMENT  0.788095    1.000000
LTU ESI         1.000000    0.736886
    EMPLOYMENT  0.736886    1.000000
LUX ESI         1.000000    0.602338
    EMPLOYMENT  0.602338    1.000000
LVA ESI         1.000000    0.905484
    EMPLOYMENT  0.905484    1.000000
NLD ESI         1.000000    0.868290
    EMPLOYMENT  0.868290    1.000000
POL ESI         1.000000    0.552877
    EMPLOYMENT  0.552877    1.000000
PRT ESI         1.000000    0.925433
    EMPLOYMENT  0.925433    1.000000
ROU ESI         1.000000    0.704159
    EMPLOYMENT  0.704159    1.000000
SVK ESI         1.000000    0.774992
    EMPLOYMENT  0.774992    1.000000
SVN ESI         1.000000    0.822135
    EMPLOYMENT  0.822135    1.000000
SWE ESI         1.000000    0.767472
    EMPLOYMENT  0.767472    1.000000

In [211]:
data_pd['ESI'].corr(data_pd['EMPLOYMENT'])

0.6798003357628429